##### Building A Chatbot

conversational RAG:- Enable a chatbot experience over an external source of data
Agents:- Build a Chatbot that can take actions

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()  ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
model = ChatGroq(model="Gemma2-9b-it",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024D53B6C1F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024D53B6CEB0>, model_name='Gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [19]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, i watched Devera and salaar movie, buti like salaar")])

AIMessage(content="It's great that you enjoyed Salaar!  \n\nIt sounds like you're really enthusiastic about it.  What did you like most about the movie? The action, the story, the characters?  \n\nI'd love to hear more about what made Salaar so enjoyable for you! 😊\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 25, 'total_tokens': 91, 'completion_time': 0.12, 'prompt_time': 0.000146989, 'queue_time': 0.014211911, 'total_time': 0.120146989}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b69f8967-686f-45a6-bf9e-ccd70917633e-0', usage_metadata={'input_tokens': 25, 'output_tokens': 66, 'total_tokens': 91})

In [20]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hello, i watched Devera and salaar movie, buti like salaa"),
        AIMessage(content='That\'s great!  \n\nIs there anything specific you liked about the movie "Devera"? \n\nFor example, did you enjoy:\n\n* **The story?**\n* **The acting?**\n* **The music?** \n* **The setting?**\n\n\nTell me more about what made you like it! 😊 \n'),
        HumanMessage(content="what movie i watched")
    ]
)

AIMessage(content='You said you watched **Devera** and **Salaar**.  \n\nWhich one did you like better?  😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 111, 'total_tokens': 139, 'completion_time': 0.050909091, 'prompt_time': 0.003516639, 'queue_time': 0.009818501, 'total_time': 0.05442573}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-31df6df3-5f12-4746-b3b0-e271e6d18342-0', usage_metadata={'input_tokens': 111, 'output_tokens': 28, 'total_tokens': 139})

### Message History
we can use message history class to wrap our model and make it stateful. This will help to keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [12]:
config= {"configurable":{"session_id":"chat1"}}

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, i watched Devera and salaar movie, but i like salaar")],
    config=config
)

In [22]:
response.content

'Ah, I see! It sounds like you had a great time watching both "Devera" and "Salaar". It\'s awesome that you have a favorite though! \n\nWhat is it about "Salaar" that you particularly enjoyed? The action, the story, the characters?  \n\nI\'m always interested to hear what people think about movies!  🎬  \n\n'

In [23]:
with_message_history.invoke(
    [HumanMessage(content="which movie i like")],
    config=config
)

AIMessage(content="You said you liked the movie **Salaar**!  \n\nIs there something specific about Salaar that you enjoyed?  Maybe a particular scene, character, or the overall story?  I'd love to hear more about what made it your favorite. 😊  \n\n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 346, 'total_tokens': 404, 'completion_time': 0.105454545, 'prompt_time': 0.010717953, 'queue_time': 0.002718247, 'total_time': 0.116172498}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1a987390-80b2-4cf8-8bd1-788cec3058a0-0', usage_metadata={'input_tokens': 346, 'output_tokens': 58, 'total_tokens': 404})

In [26]:
### change the config ----> session ID
config1= {"configurable":{"session_id":"chat2"}}
response= with_message_history.invoke(
    [HumanMessage(content="which movie i watched")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations or any information about what you've watched. \n\nTo help me figure out which movie you're thinking of, tell me some details, like:\n\n* **What genre was it?** (comedy, action, horror, etc.)\n* **When did you watch it?** (recently, a few years ago, when you were a kid?)\n* **Who were the main actors?**\n* **What was the plot about?** (even a vague description helps!)\n\n\nThe more information you give me, the better chance I have of guessing the movie! 🍿 \n"

In [30]:
response= with_message_history.invoke(
    [HumanMessage(content="MY NAME IS JAY")],
    config=config1
)
response.content

"Hi Jay! It's nice to meet you. 😄\n\nIs there anything you'd like to talk about or do today? I can tell you a story, answer your questions, or even help you write something. Just let me know!  \n"

#### Prompt Templates

prompt template help to run user info into a formate that the LLM can work with. in thsi case, the raw user input is just a message. which we are passing to LLM. lets now make that a bit more complicated. 1st lets add in a system message with some custom instructions(but still taking message as input). Next, we'll add in more input besides just the messages

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are Python developre answer the coding questions accordingly"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain= prompt|model

In [34]:
chain.invoke({"messages":[HumanMessage(content="Hi! my name is jay")]})

AIMessage(content="Hey Jay! 👋\n\nIt's great to meet you. What can I help you with today? \n\nDo you have a Python coding question you'd like me to answer? I'm ready to help you with anything from basic syntax to more complex concepts. 😊 \n\nJust provide me with the question and any relevant code snippets, and I'll do my best to assist you!  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 25, 'total_tokens': 111, 'completion_time': 0.156363636, 'prompt_time': 0.000177289, 'queue_time': 0.014231881, 'total_time': 0.156540925}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-098f7680-9b50-4270-ad5b-9686105bef9f-0', usage_metadata={'input_tokens': 25, 'output_tokens': 86, 'total_tokens': 111})

In [35]:
with_message_history= RunnableWithMessageHistory(chain, get_session_history)

In [36]:
config= {"configurable":{"session_id":"chat3"}}
response= with_message_history.invoke(
    [HumanMessage(content="Hi! my name is jay")],
    config=config
)
response

AIMessage(content="Hello Jay! \n\nIt seems like you're excited to be here!  \n\nI'm ready to answer your Python coding questions.  Just ask away, and I'll do my best to help. 😊 \n\nWhat can I help you with today? 🐍 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 25, 'total_tokens': 87, 'completion_time': 0.112727273, 'prompt_time': 0.000144359, 'queue_time': 0.01422822, 'total_time': 0.112871632}, 'model_name': 'Gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e39b3a9d-44d1-4284-9ce0-cf0e0c7fc8d2-0', usage_metadata={'input_tokens': 25, 'output_tokens': 62, 'total_tokens': 87})

In [37]:
## add more complexity

prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a python developer, answer the coding questions in {language}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt|model

In [40]:
response=chain.invoke({"messages":[HumanMessage(content="multiplication of 2 numbers")],"language":"python"})
response.content

'```python\n# Get input from the user\nnum1 = float(input("Enter the first number: "))\nnum2 = float(input("Enter the second number: "))\n\n# Calculate the product\nproduct = num1 * num2\n\n# Display the result\nprint("The product of", num1, "and", num2, "is:", product)\n```\n\n**Explanation:**\n\n1. **Get Input:**\n   - `input("Enter the first number: ")` prompts the user to enter the first number and stores it as a string.\n   - `float(...)` converts the input string to a floating-point number to handle potential decimal values.\n\n2. **Calculate Product:**\n   - `product = num1 * num2` multiplies the two numbers and stores the result in the `product` variable.\n\n3. **Display Result:**\n   - `print("The product of", num1, "and", num2, "is:", product)` prints a user-friendly message displaying the original numbers and their product.\n\n**How to Run:**\n\n1. Save the code as a `.py` file (e.g., `multiplication.py`).\n2. Open a terminal or command prompt.\n3. Navigate to the directory 

Lets now wrap this more complicated chain in a meassage history calss. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [41]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [60]:
config= {"configurable":{"session_id":"chat4"}}
response= with_message_history.invoke(
    {"messages": [HumanMessage(content="write mutliplication numbers code")], "language":"python"},
    config=config
)
response.content

'```python\ndef multiply_numbers(num1, num2):\n  """\n  Multiplies two numbers and returns the product.\n\n  Args:\n    num1: The first number.\n    num2: The second number.\n\n  Returns:\n    The product of num1 and num2.\n  """\n  return num1 * num2\n\n# Get the numbers from the user\nnum1 = float(input("Enter the first number: "))\nnum2 = float(input("Enter the second number: "))\n\n# Multiply the numbers and print the result\nproduct = multiply_numbers(num1, num2)\nprint(f"The product of {num1} and {num2} is: {product}")\n```\n\n**Explanation:**\n\n1. **`multiply_numbers(num1, num2)` Function:**\n   - Takes two numbers (`num1` and `num2`) as input.\n   - Uses the `*` operator to multiply the two numbers.\n   - Returns the result of the multiplication.\n\n2. **Main Part:**\n   - Prompts the user to enter the first and second numbers using `input()`.\n   - Converts the inputs to floating-point numbers using `float()` to allow for decimal values.\n   - Calls the `multiply_numbers()` f

#### Managing the conversation history

one important concept to understand when building chatbot is how to manage conversation history. if left unmanaged the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages'    The term "trim" often means to remove extra spaces or characters. In programming, a trim_messages function might remove unnecessary whitespace from messages or limit their length.

In [61]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens= 45,
    strategy= "last",
    token_counter= model,
    include_system=True,
    allow_partial = False,
    start_on= "human"
)

messages = [
    SystemMessage(content="you're-a good assistant"),
    HumanMessage(content="hi! Im bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 +2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),

]
trimmer.invoke(messages)

[SystemMessage(content="you're-a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 +2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [71]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain= (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what ice cream do i like?")],
    "language":"english"
    }
)
response.content

"As a large language model, I have no memory of past conversations and don't know your preferences.  \n\nWhat's your favorite ice cream flavor? 😊  \n"

In [69]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what math problem did i asked you?")],
    "language":"english"
    }
)
response.content

'You asked me "whats 2 + 2"  😊 \n'

In [72]:
### lets wrap in the message history
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat"}}

In [74]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what is my name?")],
    "language":"english"
    },
    config=config
)
response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  If you'd like to tell me your name, I'd be happy to know! 😊  \n"

In [79]:
response=with_message_history.invoke(
    {
        "messages":[HumanMessage(content="what math problem did i asked you?")],
        "language":"english",
    },
    config=config,
)
response.content

"As a large language model, I have no memory of past conversations.  \n\nPlease provide the math problem you'd like me to help with, and I'll do my best to solve it! 😊  \n\n"